## ![DLI Header](../images/DLI_Header.png)

# Anomalous Behavior Profiling (ABP)

In this notebook you are going to utilize the FIL pipeline to perform anomalous behavior profiling on GPU metric data.

## Objectives

By the time you complete this notebook you will be able to:

- Perform actual inference in a Morpheus pipeline.
- Utilize a Triton backend from a Morpheus pipeline.

---

## The Problem

Morpheus was developed at NVIDIA and in this notebook you are going to be using Morpheus to address a real cybersecurity problem that NVIDIA has had to address: the abuse of GPU resources for cryptocurrency mining.

You can imagine a massive cluster containing GPU resources that are intended for deep learning research and development. This cluster is expensive to run and maintain, and has very high demand for its use on legitimate projects. One problem is that some may see access to this GPU-enabled cluster as an opportunity to run cryptocurrency mining software for their own profit, which is obviously unethical, and blocks a scarce resource from being utilized for its intended purpose.

Consider the difficulty of solving this problem using traditional label-based methods. Cryptocurrency mining programs and process can easily have their names and outputs modified to be undetectable, and furthermore, spikes in GPU utilization are also not sufficient to identify such use since the expected workloads for the cluster are often long-running and compute-intensive.

## A Solution

Although the difference between acceptable deep learning workflows and cryptocurrency mining are very difficult to detect to the human eye, we can use AI. In this case, we can train an XGBoost model using GPU metric logs we create while running both acceptable and non-acceptable GPU workloads that we can label appropriately. The model will be capable to classify one class of workload vs. another using features that a human eye may not have been able to observe or understand on their own.

Using Morpheus, we can now load this trained model into Triton, and then construct a pipeline that ingests GPU metrics in real time and performs inference on all incoming data, labeling any data that the model classifies as participating in cryptocurrency mining.

---

## The GPU Metrics Dataset

As we have worked to develop the basics of Morpheus pipelines, we have been using `nvsmi.jsonlines`. But now that we are going to do meaningful work with it, let's take a closer look.

`nvsmi.jsonlines` is known to contain mining behavior profiles. The dataset is in the `.jsonlines` format which means each new line represents an new JSON object. In order to parse this data, it must be ingested, split by lines into individual JSON objects, and parsed into cuDF dataframes. This will all be handled by Morpheus.

The dataset contains NVIDIA GPU metrics at regular time intervals and was extracted by a [NetQ](https://www.nvidia.com/en-us/networking/ethernet-switching/netq/) agent and serialized into JSON. Each line in the dataset contains much of the same information that is returned by the `nvidia-smi` utility. For those unfamiliar, `nvidia-smi` is a command-line program to inspect how GPUs in a system are being utilized. If you were to run `nvidia-smi dmon` yourself, you would see output similar to the following:

```sh
$ nvidia-smi dmon
# gpu   pwr gtemp mtemp    sm   mem   enc   dec  mclk  pclk
# Idx     W     C     C     %     %     %     %   MHz   MHz
    0    70    48     -     5     1     0     0  7000  1350
    0    68    48     -    11     1     0     0  7000  1350
    0    69    48     -     3     1     0     0  7000  1350
    0   270    53     -    10     1     0     0  7000  1875
    0   274    55     -    75    46     0     0  7000  1740
    0   278    55     -    86    56     0     0  7000  1755
    0   279    56     -    99    63     0     0  7000  1755
    0   277    57     -    86    55     0     0  7000  1755
    0   281    57     -    85    54     0     0  7000  1740
```

Each line in the output represents the GPU metrics at a single point in time. As the tool progresses the GPU begins to be utilized and you can see the `SM%` and `Mem%` increase as memory is loaded into the GPU and computations are performed.

The model we will be using can ingest this information and determine whether or not the GPU is mining cryptocurrencies without needing additional information from the host machine.

**Optional:** Feel free if you like to open a terminal and issue the `nvidia-smi dmon` command. Your output will be significantly different than the above on account of the fact that you are not currently utilizing the available GPU in this environment.

### Data Exploration

Before building out the pipeline, let's look briefly at the dataset.

In [21]:
import pandas as pd

In [22]:
source = pd.read_json('nvsmi.jsonlines', lines=True)

In [23]:
source.shape

(1242, 175)

In [24]:
source.dtypes

nvidia_smi_log.timestamp                               object
nvidia_smi_log.gpu.pci.tx_util                         object
nvidia_smi_log.gpu.pci.rx_util                         object
nvidia_smi_log.gpu.fb_memory_usage.used                object
nvidia_smi_log.gpu.fb_memory_usage.free                object
                                                    ...      
nvidia_smi_log.gpu.utilization.decoder_util            object
nvidia_smi_log.gpu.utilization.encoder_util            object
nvidia_smi_log.gpu.uuid                                object
nvidia_smi_log.gpu.vbios_version                       object
timestamp                                      datetime64[ns]
Length: 175, dtype: object

In [25]:
source.head()

,nvidia_smi_log.timestamp,nvidia_smi_log.gpu.pci.tx_util,nvidia_smi_log.gpu.pci.rx_util,nvidia_smi_log.gpu.fb_memory_usage.used,nvidia_smi_log.gpu.fb_memory_usage.free,nvidia_smi_log.gpu.bar1_memory_usage.total,nvidia_smi_log.gpu.bar1_memory_usage.used,nvidia_smi_log.gpu.bar1_memory_usage.free,nvidia_smi_log.gpu.utilization.gpu_util,nvidia_smi_log.gpu.utilization.memory_util,...,nvidia_smi_log.gpu.retired_pages.pending_retirement,nvidia_smi_log.gpu.serial,nvidia_smi_log.gpu.supported_gpu_target_temp.gpu_target_temp_max,nvidia_smi_log.gpu.supported_gpu_target_temp.gpu_target_temp_min,nvidia_smi_log.gpu.temperature.gpu_target_temperature,nvidia_smi_log.gpu.utilization.decoder_util,nvidia_smi_log.gpu.utilization.encoder_util,nvidia_smi_log.gpu.uuid,nvidia_smi_log.gpu.vbios_version,timestamp
0,Fri Mar 12 09:46:00 2021,0 KB/s,0 KB/s,3909 MiB,12251 MiB,16384 MiB,7 MiB,16377 MiB,100 %,2 %,...,No,322917026071,N/A,N/A,N/A,0 %,0 %,GPU-98f29bfe-f021-7214-6f7a-b70c56ecf2aa,88.00.4F.00.09,2021-03-12 09:46:00.956650240
1,Fri Mar 12 09:46:31 2021,0 KB/s,2000 KB/s,3909 MiB,12251 MiB,16384 MiB,7 MiB,16377 MiB,100 %,5 %,...,No,322917026071,N/A,N/A,N/A,0 %,0 %,GPU-98f29bfe-f021-7214-6f7a-b70c56ecf2aa,88.00.4F.00.09,2021-03-12 09:46:31.070379008
2,Fri Mar 12 09:47:01 2021,1000 KB/s,2000 KB/s,3909 MiB,12251 MiB,16384 MiB,7 MiB,16377 MiB,100 %,5 %,...,No,322917026071,N/A,N/A,N/A,0 %,0 %,GPU-98f29bfe-f021-7214-6f7a-b70c56ecf2aa,88.00.4F.00.09,2021-03-12 09:47:01.904752128
3,Fri Mar 12 09:47:32 2021,0 KB/s,2000 KB/s,3909 MiB,12251 MiB,16384 MiB,7 MiB,16377 MiB,100 %,6 %,...,No,322917026071,N/A,N/A,N/A,0 %,0 %,GPU-98f29bfe-f021-7214-6f7a-b70c56ecf2aa,88.00.4F.00.09,2021-03-12 09:47:32.742011904
4,Fri Mar 12 09:48:02 2021,1000 KB/s,2000 KB/s,3909 MiB,12251 MiB,16384 MiB,7 MiB,16377 MiB,100 %,6 %,...,No,322917026071,N/A,N/A,N/A,0 %,0 %,GPU-98f29bfe-f021-7214-6f7a-b70c56ecf2aa,88.00.4F.00.09,2021-03-12 09:48:02.556625152


---

## Add Triton Inference

Let's build inference into the pipeline that will use the `abp-nvsmi-xgb` XGBoost model, already loaded into Triton, as shown in the previous notebook.

To begin, here is the pipeline we have build out so far which only performs no-op inference:

```sh
morpheus run pipeline-fil \
  from-file --filename=nvsmi.jsonlines \
  deserialize \
  preprocess \
  inf-identity \
  serialize \
  to-file --filename=output.jsonlines
```

Here we replace the `inf-identity` stage with `inf-triton`:

In [6]:
!morpheus run pipeline-fil \
  from-file --filename=nvsmi.jsonlines \
  deserialize \
  preprocess \
  inf-triton \
  serialize \
  to-file --filename=output.jsonlines

Configuring Pipeline via CLI
Usage: morpheus run pipeline-fil inf-triton [OPTIONS]
Try 'morpheus run pipeline-fil inf-triton --help' for help.

Error: Missing option '--model_name'.


We can see that we are missing the `--model_name` option for the `inf-triton` stage. Let's look at the help message for this stage as recommended:

In [7]:
!morpheus run pipeline-fil inf-triton --help

Configuring Pipeline via CLI
Usage: morpheus run pipeline-fil inf-triton [OPTIONS]

Options:
  --model_name TEXT               Model name in Triton to send messages to
                                  [required]
  --server_url TEXT               Triton server URL (IP:Port)  [required]
  --force_convert_inputs BOOLEAN  Instructs this stage to forcibly convert all
                                  input types to match what Triton is
                                  expecting. Even if this is set to `False`,
                                  automatic conversion will be done only if
                                  there would be no data loss (i.e. int32 ->
                                  int64).  [default: False]
  --use_shared_memory BOOLEAN     Whether or not to use CUDA Shared IPC Memory
                                  for transferring data to Triton. Using CUDA
                                  IPC reduces network transfer time but
                                  requires th

The message indicates that there are two required options we need to provide: `--model_name` and `--server_url`.

---

## Exercise Part 1: Send Data to `abp-nvsmi-xgb` in Triton

Fix the `inf-triton` stage in the pipeline below to send data for inference to the `abp-nvsmi-xgb` model, already loaded for you into the Triton server running at `triton:8001`.

In [9]:
!morpheus run pipeline-fil \
  from-file --filename=nvsmi.jsonlines \
  deserialize \
  preprocess \
  inf-triton --model_name="abp-nvsmi-xgb" --server_url="triton:8001" \
  serialize \
  to-file --filename=output.jsonlines

Configuring Pipeline via CLI
Starting pipeline via CLI... Ctrl+C to Quit


#### Solution

If you get stuck or need help, click the `...` below to display the solution.

In [ ]:
!morpheus run pipeline-fil \
  from-file --filename=nvsmi.jsonlines \
  deserialize \
  preprocess \
  inf-triton --model_name="abp-nvsmi-xgb" --server_url="triton:8001" \
  serialize \
  to-file --filename=output.jsonlines

---

### Check for Results

Assuming the pipeline has run without errors, we would like to see the results of the inference. Let's take a quick look at the first row of our results:

In [10]:
import pandas as pd

In [11]:
results = pd.read_json('output.jsonlines', lines=True)

In [12]:
results.head(1)

,nvidia_smi_log.timestamp,nvidia_smi_log.gpu.pci.tx_util,nvidia_smi_log.gpu.pci.rx_util,nvidia_smi_log.gpu.fb_memory_usage.used,nvidia_smi_log.gpu.fb_memory_usage.free,nvidia_smi_log.gpu.bar1_memory_usage.total,nvidia_smi_log.gpu.bar1_memory_usage.used,nvidia_smi_log.gpu.bar1_memory_usage.free,nvidia_smi_log.gpu.utilization.gpu_util,nvidia_smi_log.gpu.utilization.memory_util,...,nvidia_smi_log.gpu.retired_pages.pending_retirement,nvidia_smi_log.gpu.serial,nvidia_smi_log.gpu.supported_gpu_target_temp.gpu_target_temp_max,nvidia_smi_log.gpu.supported_gpu_target_temp.gpu_target_temp_min,nvidia_smi_log.gpu.temperature.gpu_target_temperature,nvidia_smi_log.gpu.utilization.decoder_util,nvidia_smi_log.gpu.utilization.encoder_util,nvidia_smi_log.gpu.uuid,nvidia_smi_log.gpu.vbios_version,timestamp
0,Fri Mar 12 09:46:00 2021,0,0,3909,12251,16384,7,16377,100,2,...,No,322917026071,N/A,N/A,N/A,0 %,0 %,GPU-98f29bfe-f021-7214-6f7a-b70c56ecf2aa,88.00.4F.00.09,2021-03-12 09:46:00.956650496


Looking at the output, we do not see any indication that inference has been performed. The reason is simple, we need to explicitly add the results by adding an `add-class` stage to the pipeline:

In [13]:
!morpheus run pipeline-fil --help | grep 'add-class'

  4. The following stages must come after an inference stage: `add-class`, `filter`, `gen-viz`
  add-class     Add detected classifications to each message


We can see that `add-class` will add detected classifications to each message, and that the `add-class` stage needs to come after we perform inference.

---

## Exercise Part 2: Add Inference Results to the Pipeline

Fix the pipeline below to add classification labels to each message.

In [14]:
!morpheus run pipeline-fil \
  from-file --filename=nvsmi.jsonlines \
  deserialize \
  preprocess \
  inf-triton --model_name="abp-nvsmi-xgb" --server_url="triton:8001" --force_convert_inputs=True \
  add-class \
  serialize \
  to-file --filename=output.jsonlines --overwrite

Configuring Pipeline via CLI
Starting pipeline via CLI... Ctrl+C to Quit


#### Solution

If you get stuck or need help, click the `...` below to display the solution.

In [ ]:
!morpheus run pipeline-fil \
  from-file --filename=nvsmi.jsonlines \
  deserialize \
  preprocess \
  inf-triton --model_name="abp-nvsmi-xgb" --server_url="triton:8001" --force_convert_inputs=True \
  add-class \
  serialize \
  to-file --filename=output.jsonlines --overwrite

---

### Check for Results

In [15]:
import pandas as pd

In [16]:
results = pd.read_json('output.jsonlines', lines=True)

In [17]:
results.dtypes

nvidia_smi_log.timestamp                               object
nvidia_smi_log.gpu.pci.tx_util                          int64
nvidia_smi_log.gpu.pci.rx_util                          int64
nvidia_smi_log.gpu.fb_memory_usage.used                 int64
nvidia_smi_log.gpu.fb_memory_usage.free                 int64
                                                    ...      
nvidia_smi_log.gpu.utilization.encoder_util            object
nvidia_smi_log.gpu.uuid                                object
nvidia_smi_log.gpu.vbios_version                       object
timestamp                                      datetime64[ns]
mining                                                  int64
Length: 176, dtype: object

We can see that an `int64` `mining` column has been added to each row of the data. The `add-class` stage will set the value of this field to `1` for each row that was identified during inference as being involved in cryptocurrency mining.

In [18]:
mining = results[results['mining'] == 1]

In [19]:
mining.shape

(188, 176)

In [20]:
mining.head()

,nvidia_smi_log.timestamp,nvidia_smi_log.gpu.pci.tx_util,nvidia_smi_log.gpu.pci.rx_util,nvidia_smi_log.gpu.fb_memory_usage.used,nvidia_smi_log.gpu.fb_memory_usage.free,nvidia_smi_log.gpu.bar1_memory_usage.total,nvidia_smi_log.gpu.bar1_memory_usage.used,nvidia_smi_log.gpu.bar1_memory_usage.free,nvidia_smi_log.gpu.utilization.gpu_util,nvidia_smi_log.gpu.utilization.memory_util,...,nvidia_smi_log.gpu.serial,nvidia_smi_log.gpu.supported_gpu_target_temp.gpu_target_temp_max,nvidia_smi_log.gpu.supported_gpu_target_temp.gpu_target_temp_min,nvidia_smi_log.gpu.temperature.gpu_target_temperature,nvidia_smi_log.gpu.utilization.decoder_util,nvidia_smi_log.gpu.utilization.encoder_util,nvidia_smi_log.gpu.uuid,nvidia_smi_log.gpu.vbios_version,timestamp,mining
1054,Fri Mar 12 18:58:29 2021,72000,103000,7264,8896,16384,11,16373,100,17,...,322917026071,N/A,N/A,N/A,0 %,0 %,GPU-98f29bfe-f021-7214-6f7a-b70c56ecf2aa,88.00.4F.00.09,2021-03-12 18:58:29.392179968,1
1055,Fri Mar 12 18:59:01 2021,77000,94000,7264,8896,16384,11,16373,100,18,...,322917026071,N/A,N/A,N/A,0 %,0 %,GPU-98f29bfe-f021-7214-6f7a-b70c56ecf2aa,88.00.4F.00.09,2021-03-12 18:59:01.259411200,1
1056,Fri Mar 12 18:59:32 2021,73000,86000,7264,8896,16384,11,16373,100,17,...,322917026071,N/A,N/A,N/A,0 %,0 %,GPU-98f29bfe-f021-7214-6f7a-b70c56ecf2aa,88.00.4F.00.09,2021-03-12 18:59:32.822284544,1
1057,Fri Mar 12 19:00:03 2021,49000,101000,7264,8896,16384,11,16373,100,19,...,322917026071,N/A,N/A,N/A,0 %,0 %,GPU-98f29bfe-f021-7214-6f7a-b70c56ecf2aa,88.00.4F.00.09,2021-03-12 19:00:03.437993984,1
1058,Fri Mar 12 19:00:35 2021,58000,83000,7264,8896,16384,11,16373,100,18,...,322917026071,N/A,N/A,N/A,0 %,0 %,GPU-98f29bfe-f021-7214-6f7a-b70c56ecf2aa,88.00.4F.00.09,2021-03-12 19:00:35.062054144,1


---

## Next

In the next section we will begin to learn about a second Morpheus pipeline, `pipeline-nlp` first with a very high-level introduction to natural language processing.

Please continue to the next notebook.